In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import re

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
results_root_path = "ssvep-bci-project/results/scripts_new"
script_list = os.listdir(results_root_path)

In [ ]:
script_list.sort()

In [ ]:
script_list

In [ ]:
acc_list = []

for script in script_list:
    runs = os.listdir(f"{results_root_path}/{script}")
    for run in runs:
        acc = np.load(f"{results_root_path}/{script}/{run}/accuracy.npy")
        acc_list.append(
            pd.DataFrame(acc, columns=["correct_predictions", "accuracy"])
            .assign(script=script, run=run)
            .reset_index()
            .rename(columns={"index": "subject"})
        )
    
dataframe = pd.concat(acc_list)

In [ ]:
def add_delta_tau_to_dataframe(df):
    
    return df.assign(
        delta=df["run"].apply(lambda x: int(re.findall(r'\d+', x)[0])),
        tau=df["run"].apply(lambda x: int(re.findall(r'\d+', x)[1]))
    )

## Experimento 1

In [ ]:
res_ss_cca = (
    dataframe
    .groupby(["script", "run",
              "subject"
             ]).agg({
        "accuracy": "mean",
        "correct_predictions": "sum",
    })
    .loc["ss_cca_hp_gridsearch_harmonics_equalized"]
    .sort_values(by="accuracy", ascending=False)
    .reset_index()
    .pipe(add_delta_tau_to_dataframe)
)

In [ ]:
res_ss_cca.groupby(["run", "tau", "delta"])["accuracy"].mean().reset_index().sort_values(by="accuracy", ascending=False)

In [ ]:
# plt.figure(figsize=(8,4))
ax = sns.lineplot(
    data=res_ss_cca.sort_values("delta"),
    x="delta",
    y="accuracy"
)
ax.set_ylabel("Acurácia")
ax.set_xlabel("Atraso do filtro $\delta$")
ax.set_title("$\delta$ vs. Acurácia - Algoritmo SS-CCA Correlação")
sns.despine()

plt.savefig("images/delta_acc_ss_cca_correlacao.svg", format="svg", dpi=2000)
plt.show()

In [ ]:
res_ss_cca_fixed = (
    dataframe
    .groupby(["script", "run",
              "subject"
             ]).agg({
        "accuracy": "mean",
        "correct_predictions": "sum",
    })
    .loc["ss_cca_fixed_hp_gridsearch_harmonics_equalized"]
    .sort_values(by="accuracy", ascending=False)
    .reset_index()
    .pipe(add_delta_tau_to_dataframe)
)

In [ ]:
res_ss_cca_fixed.groupby(["run", "tau", "delta"])["accuracy"].mean().reset_index().sort_values(by="accuracy", ascending=False)

In [ ]:
# plt.figure(figsize=(8,4))
ax = sns.lineplot(
    data=res_ss_cca_fixed.sort_values("delta"),
    x="delta",
    y="accuracy",
    errorbar=("se", 2)
)
ax.set_ylabel("Acurácia")
ax.set_xlabel("Atraso do filtro $\delta$")
ax.set_title("$\delta$ vs. Acurácia - Algoritmo SS-CCA Filtro")
sns.despine()

plt.savefig("images/delta_acc_ss_cca_filtro.svg", format="svg", dpi=2000)
plt.show()

In [ ]:
res_fir_cca_all = (
    dataframe
    .groupby(["script", "run",
              "subject"
             ]).agg({
        "accuracy": "mean",
        "correct_predictions": "sum",
    })
    .loc[["ss_cca_hp_gridsearch_harmonics_equalized", "ss_cca_fixed_hp_gridsearch_harmonics_equalized"]]
    .sort_values(by="accuracy", ascending=False)
    .reset_index()
    .pipe(add_delta_tau_to_dataframe)
    [lambda x: x["tau"]<=30]
    .assign(Algoritmo=lambda x: x["script"].map({
        "ss_cca_hp_gridsearch_harmonics_equalized": "SS-CCA Correlação",
        "ss_cca_fixed_hp_gridsearch_harmonics_equalized": "SS-CCA Projeção",
    }))
)

g = sns.relplot(
    data=res_fir_cca_all.sort_values(["delta", "script"]),
    x="delta",
    y="accuracy",
    col="Algoritmo",
    col_order=["SS-CCA Correlação", "SS-CCA Projeção"],
    errorbar=("se", 2),
    kind="line",
    facet_kws={"sharey": False}
)

g.set_ylabels("Acurácia")
g.set_xlabels("Atraso do filtro $\delta$")
g.fig.suptitle("$\delta$ vs. Acurácia - Algoritmo SS-CCA", fontsize=16)
g.fig.subplots_adjust(top=.85)
sns.despine()

plt.savefig("images/delta_acc_ss_cca_all.svg", format="svg", dpi=2000)
plt.show()

## Experimento 2

In [ ]:
res_fir_cca = (
    dataframe
    .groupby(["script", "run",
              "subject"
             ]).agg({
        "accuracy": "mean",
        "correct_predictions": "sum",
    })
    .loc["fir_cca_hp_gridsearch_harmonics_equalized"]
    .sort_values(by="accuracy", ascending=False)
    .reset_index()
    .pipe(add_delta_tau_to_dataframe)
)

In [ ]:
res_fir_cca.groupby(["run", "tau", "delta"])["accuracy"].mean().reset_index().sort_values(by="accuracy", ascending=False)

In [ ]:
# plt.figure(figsize=(8,4))
ax = sns.lineplot(
    data=res_fir_cca.sort_values("delta"),
    x="tau",
    y="accuracy",
    errorbar=("se", 2)
)
ax.set_ylabel("Acurácia")
ax.set_xlabel("Tamanho do filtro $\u03C4$")
ax.set_title("$\u03C4$ vs. Acurácia - Algoritmo SS-CCA Correlação")
sns.despine()

plt.savefig("images/tau_acc_fir_cca_correlacao.svg", format="svg", dpi=2000)
plt.show()

In [ ]:
res_fir_cca_fixed = (
    dataframe
    .groupby(["script", "run",
              "subject"
             ]).agg({
        "accuracy": "mean",
        "correct_predictions": "sum",
    })
    .loc["fir_cca_fixed_hp_gridsearch_harmonics_equalized"]
    .sort_values(by="accuracy", ascending=False)
    .reset_index()
    .pipe(add_delta_tau_to_dataframe)
)

In [ ]:
res_fir_cca_fixed.groupby(["run", "tau", "delta"])["accuracy"].mean().reset_index().sort_values(by="accuracy", ascending=False)

In [ ]:
ax = sns.lineplot(
    data=res_fir_cca_fixed.sort_values("delta"),
    x="tau",
    y="accuracy",
    errorbar=("se", 2)
)
ax.set_ylabel("Acurácia")
ax.set_xlabel("Tamanho do filtro $\u03C4$")
ax.set_title("$\u03C4$ vs. Acurácia - Algoritmo SS-CCA Filtro")
sns.despine()

plt.savefig("images/tau_acc_fir_cca_filtro.svg", format="svg", dpi=2000)
plt.show()

In [ ]:
res_fir_cca_all = (
    dataframe
    .groupby(["script", "run",
              "subject"
             ]).agg({
        "accuracy": "mean",
        "correct_predictions": "sum",
    })
    .loc[["fir_cca_hp_gridsearch_harmonics_equalized", "fir_cca_fixed_hp_gridsearch_harmonics_equalized"]]
    .sort_values(by="accuracy", ascending=False)
    .reset_index()
    .pipe(add_delta_tau_to_dataframe)
    [lambda x: x["tau"]<=30]
    .assign(Algoritmo=lambda x: x["script"].map({
        "fir_cca_hp_gridsearch_harmonics_equalized": "FIR-CCA Correlação",
        "fir_cca_fixed_hp_gridsearch_harmonics_equalized": "FIR-CCA Projeção",
    }))
)

g = sns.relplot(
    data=res_fir_cca_all.sort_values(["delta", "script"]),
    x="tau",
    y="accuracy",
    col="Algoritmo",
    col_order=["FIR-CCA Correlação", "FIR-CCA Projeção"],
    errorbar=("se", 2),
    kind="line",
    facet_kws={"sharey": False}
)

g.set_ylabels("Acurácia")
g.set_xlabels("Tamanho do filtro $\u03C4$")
g.fig.suptitle("$\u03C4$ vs. Acurácia - Algoritmo FIR-CCA", fontsize=16)
g.fig.subplots_adjust(top=.85)
sns.despine()

plt.savefig("images/tau_acc_fir_cca_all.svg", format="svg", dpi=2000)
plt.show()

## Experimento 3

In [ ]:
overall_2s = (
    dataframe.groupby(["script", "run"]).agg({
        "accuracy": "mean",
        "correct_predictions": "sum",
    })
    .sort_values(by="accuracy", ascending=False)
    .loc["overall_benchmark_harmonics_equalized"].sort_index()
)

In [ ]:
overall_2s = overall_2s.reset_index()

In [ ]:
overall_2s["Configuracao"] = overall_2s["run"].str.contains("Fixed").map({False: "Correlação",
                                                                          True: "Projeção"})

overall_2s["Acurácia"] = overall_2s["accuracy"].round(3)

def clean_run_name(s):
    l = s.replace("_Fixed", "").split("_")[1:-1]
    return "_".join(l)

overall_2s["Algoritmo"] = overall_2s["run"].apply(clean_run_name)

In [ ]:
overall_2s = overall_2s.set_index(["Configuracao", "Algoritmo"])["Acurácia"].unstack(0).iloc[[
    0, 5, 4, 1, 3, 2
]]
overall_2s

In [ ]:
# overall_2s.columns = [["Configuracao", "Configuracao"], ["Correlação", "Filtro"]]
overall_2s.index = [
    "CCA Default",
    "SS-CCA (delta=3)",
    "FIR-CCA (delta=0, tau=5)",
    "FB-CCA",
    "SS-FB-CCA (delta=3)",
    "FIR-FB-CCA (delta=0, tau=5)",
]
overall_2s.index.name = "Algoritmo"

overall_2s

In [ ]:
print(overall_2s.to_latex(decimal=","))
overall_2s

In [ ]:
dataframe.groupby(["script", "run"]).agg({
    "accuracy": "mean",
    "correct_predictions": "sum",
}).sort_values(by="accuracy", ascending=False).loc["overall_benchmark_3s"].sort_index()

In [ ]:
dataframe.groupby(["script", "run"]).agg({
    "accuracy": "mean",
    "correct_predictions": "sum",
}).sort_values(by="accuracy", ascending=False).loc["overall_benchmark_harmonics_equalized"].sort_index()

# Anotações

CCA
- numero de harmonicas
- tamanho da janela
- lista de eletrodos

Spatial:
- tamanho do passo
- tamanho da janela
+ experimentar fazer a curva de desempenho vs tamanho da janela, fixando o passo em 0
+ explicar que estamos usando os primeiro 500ms para preencher o filtro FIR das primeiras amostras

Filter bank:
- número de filtros
- hiperparâmetros do kernel arbitrário

## Revisão bibliográfica

- Artigo 2011: algoritmo que detecta a fase
- Artigo de 2014a: usa o EEG de treino averaged como um sinal de referência para o CCA
- Artigo de 2014b: utiliza o CCA como estimador de um filtro espacial que é utilizado em seguida (identico ao que estamos chamando de fixed).


## Experimentos

Sempre janelas de 2 segundos

Hiperparametros:

- SS
    - testes vários gaps, sempre com janela de tamanho 1
    - treinando e sem treino
- FIR
    - gap = 0, mas vários tamanhos de janela
    - treinando e sem treino


CCA-correlation

- CCA Original
- SS-CCA Original (PEGAR HIPERPARAMETRO DO ARTIGO)
- CCA FIR
- FB-CCA
- FB-CCA SS
- FB-CCA FIR


Depois com fixed

- CCA Original (with training)
- SS-CCA Original (PEGAR HIPERPARAMETRO DO ARTIGO)
- CCA FIR (with training) ==> testar janelas diferentes, variar até algum máximo
- FB-CCA (with training)
- FB-CCA with FIR-filter (with training)


## Ficou faltando:

- Utilizar o dado de treinamento inteiro (sem filtrar os primeiros segundos)

Qualificação até final de setembro
- 15 dias de antecedência pra entrar com pedido
- teria que estar pronto no dia que abre o pedido
- olhar instruções na página da pós-graduação da FEEC
- aluno que marca
- https://www.fee.unicamp.br/pos-graduacao/alunos-regulares/
    > vida academica
    > >exame de qualificação

Proposta:

- Uso de mais atrasos no vetor de variáveis aleatórias
    - Expandindo o algoritmo proposto Spatio-temporal CCA
    - É como se fosse um "banco de filtros FIR"
- Contraste entre aplicação do CCA janela a janela vs. CCA com a fase e vetor de pesos fixo:
    - Fazer uma revisão dos artigos citados no Artigo de 2015
    - Subproposta: combinar FB-CCA com CCA pesos fixos para maximizar performance

Conclusão: o que está feito e o que falta fazer

- Ajuste do kernel linear que combina múltiplos processos CCA numa correlação final

- No texto, descrever os processos via diagrama
- Primeiramente colocar os resultados e ver quanto espaço sobra
- Citar o trabalho para explicar os modelos CCA que usam dado de treinamento e
    - discutir que essa abordagem faz uso da fase
    - por isso é necessario que o sinal de referência tenha sempre a mesma diferença de fase inicial em relação a estimulação
- Priorizar explicar o CCA
- Os outros algoritmos podem ser explicados de maneira mais simples

Próximos passos:

- Utilizar dados dos 70 candidatos da base chinesa
- Usar dados coletados no lab
- Publicação em revista